In [1]:
using LinearAlgebra, DataFrames, CSV, Query, Random, StatsBase, Statistics

# Dataprep

## Reading the Data

Reading in the CSV-file which i acquired from Kaggle. 

In [2]:
cc = CSV.read("Data/UCI_Credit_Card.csv")
cols = names(cc)
println(describe(cc))

25×8 DataFrame
│ Row │ variable                   │ mean      │ min       │ median   │ max       │ nunique │ nmissing │ eltype   │
│     │ Symbol                     │ Float64   │ Real      │ Float64  │ Real      │ Nothing │ Nothing  │ DataType │
├─────┼────────────────────────────┼───────────┼───────────┼──────────┼───────────┼─────────┼──────────┼──────────┤
│ 1   │ ID                         │ 15000.5   │ 1         │ 15000.5  │ 30000     │         │          │ Int64    │
│ 2   │ LIMIT_BAL                  │ 1.67484e5 │ 10000.0   │ 140000.0 │ 1.0e6     │         │          │ Float64  │
│ 3   │ SEX                        │ 1.60373   │ 1         │ 2.0      │ 2         │         │          │ Int64    │
│ 4   │ EDUCATION                  │ 1.85313   │ 0         │ 2.0      │ 6         │         │          │ Int64    │
│ 5   │ MARRIAGE                   │ 1.55187   │ 0         │ 2.0      │ 3         │         │          │ Int64    │
│ 6   │ AGE                        │ 35.4855   │ 21      

## Inspection & Preparation of the Data

### SEX

This variable is categorical. 

We have `1 = Male` and `2 = Female`.

I map this to `0 = Male` and `1 = Female`.

In [3]:
unique(cc[:SEX])

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:1
└ @ Core In[3]:1


2-element Array{Int64,1}:
 2
 1

In [4]:
cc[:SEX] = map(s -> ifelse(s == 1, 0, 1), cc.SEX)

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[4]:1
└ @ Core In[4]:1


30000-element Array{Int64,1}:
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

### EDUCATION

According to the UCI post where they explain the data, the coding for this variable is as follows:
* `1 = graduate school`
* `2 = university`
* `3 = high school`
* `4 = others`

But there are some additional values that can be found in the data, which are not explained by the researchers. I map this variable in the following way.
* `1 = graduate school`
* `2 = university`
* `3 = high school`
* `4 = others`
* `! in 1, 2, 3, 4 => 5 = unknown`

This data is ordinal, i.e. it is possible to rank but not determine relative "value" of one category compared to another.

In [5]:
sort(unique(cc.EDUCATION))

7-element Array{Int64,1}:
 0
 1
 2
 3
 4
 5
 6

In [6]:
cc[:EDUCATION] = map(s -> ifelse(s in (1, 2, 3, 4), s, 5), cc.EDUCATION)
sort(unique(cc.EDUCATION))

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[6]:1
└ @ Core In[6]:1


5-element Array{Int64,1}:
 1
 2
 3
 4
 5

### Marriage

According to UCI the coding for this variable is as follows:
* `1 = married`
* `2 = single`
* `3 = others`

There is an unknown value here as well, but since there are not that many marital statuses, I add them to the `others` category.
* `1 = married`
* `2 = single`
* `3 = others`
* `! in 1, 2, 3 => 3 = others`

This data is categorical.

In [7]:
sort(unique(cc.MARRIAGE))

4-element Array{Int64,1}:
 0
 1
 2
 3

In [8]:
cc[:MARRIAGE] = map(s -> ifelse(s in (1, 2, 3), s, 3), cc.MARRIAGE)
sort(unique(cc.MARRIAGE))

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1


3-element Array{Int64,1}:
 1
 2
 3

### Pay

The variables PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, and PAY_6 supposedly have the following mapping:
* `-1 = pay duly`
* `1 = 1 mnth over due`
* `2 = 2 mnth over due`
* ...
* `8 = 8 mnth over due`
* `9 = 9 mnth or more over due`

But there are some unknown values to be found for these variables also. Since these are ordinal variables, and since it is justifiable to treat them as interval data (i.e. that being 2 months over due is twice the amount of time past due as someone being 1 month over due), I assume that the values `-2`, `-1` and `0` all mean `pay duly` and map them to a new label `0`. 

In the logreg model later, this gives the interpretation that the coefficients for the PAY variable change the predicted logodds of default by twice the amount if someone is 2 months behind compared to 1 month, and 0 if they pay duly. 

In [9]:
for i in [0, 2, 3, 4, 5, 6]
    #println(Symbol("PAY_", i))
    cc[Symbol("PAY_", i)] = map(s -> ifelse(s in (-2, -1), 0, s),
                                cc[Symbol("PAY_", i)])
end

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[9]:3
└ @ Core In[9]:3
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[9]:3
└ @ Core In[9]:3


#### Inspection after cleaning

In [10]:
println(describe(cc))

25×8 DataFrame
│ Row │ variable                   │ mean      │ min       │ median   │ max       │ nunique │ nmissing │ eltype   │
│     │ Symbol                     │ Float64   │ Real      │ Float64  │ Real      │ Nothing │ Nothing  │ DataType │
├─────┼────────────────────────────┼───────────┼───────────┼──────────┼───────────┼─────────┼──────────┼──────────┤
│ 1   │ ID                         │ 15000.5   │ 1         │ 15000.5  │ 30000     │         │          │ Int64    │
│ 2   │ LIMIT_BAL                  │ 1.67484e5 │ 10000.0   │ 140000.0 │ 1.0e6     │         │          │ Float64  │
│ 3   │ SEX                        │ 0.603733  │ 0         │ 1.0      │ 1         │         │          │ Int64    │
│ 4   │ EDUCATION                  │ 1.85377   │ 1         │ 2.0      │ 5         │         │          │ Int64    │
│ 5   │ MARRIAGE                   │ 1.55727   │ 1         │ 2.0      │ 3         │         │          │ Int64    │
│ 6   │ AGE                        │ 35.4855   │ 21      

## One-Hot Encoding

The following variables are either categorical or ordinal without being suited to be treated as interval variables:
* `SEX`
* `EDUCATION`
* `MARRIAGE`

By coding `SEX` as a binary variable, it is not necessary to perform one-hot encoding of this variable, since 0 = Male becomes the reference, and the coefficient for `SEX` is interpreted as the exposure relative to the reference, and only adds to the logodds when `SEX` = 1 = Female. 

For `EDUCATION` and `MARRIAGE`, one-hot encoding is necessary. 


In [11]:
function OneHot(x::Vector{Int64}, lab::Symbol)
    p = sort(unique(x))
    colname = string(lab)
    OHdf = DataFrame(keys = p)

    for i in 1:length(p)
        val = p[i]
        v = map(s -> ifelse(s == val, 1, 0), OHdf.keys)
        insertcols!(OHdf, i+1, Symbol(colname, val) => v)
    end
    OHdf
end

OneHot (generic function with 1 method)

In [12]:
# Education variable
edu = Vector(cc[:EDUCATION])
lab = :EDUCATION
M1 = OneHot(edu, lab)
M1

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[12]:1
└ @ Core In[12]:1


,keys,EDUCATION1,EDUCATION2,EDUCATION3,EDUCATION4,EDUCATION5
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,0,0,0,0
2,2,0,1,0,0,0
3,3,0,0,1,0,0
4,4,0,0,0,1,0
5,5,0,0,0,0,1


In [13]:
# Marriage variable
marriage = Vector(cc[:MARRIAGE])
lab = :MARRIAGE
M2 = OneHot(marriage, lab)
M2

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[13]:1
└ @ Core In[13]:1


,keys,MARRIAGE1,MARRIAGE2,MARRIAGE3
,Int64,Int64,Int64,Int64
1,1,1,0,0
2,2,0,1,0
3,3,0,0,1


#### Adding back into the dataframe

In [14]:
cc = join(cc, M1, on = :EDUCATION => :keys)

println(describe(cc))

30×8 DataFrame
│ Row │ variable                   │ mean      │ min       │ median   │ max       │ nunique │ nmissing │ eltype   │
│     │ Symbol                     │ Float64   │ Real      │ Float64  │ Real      │ Nothing │ Nothing  │ DataType │
├─────┼────────────────────────────┼───────────┼───────────┼──────────┼───────────┼─────────┼──────────┼──────────┤
│ 1   │ ID                         │ 15000.5   │ 1         │ 15000.5  │ 30000     │         │          │ Int64    │
│ 2   │ LIMIT_BAL                  │ 1.67484e5 │ 10000.0   │ 140000.0 │ 1.0e6     │         │          │ Float64  │
│ 3   │ SEX                        │ 0.603733  │ 0         │ 1.0      │ 1         │         │          │ Int64    │
│ 4   │ EDUCATION                  │ 1.85377   │ 1         │ 2.0      │ 5         │         │          │ Int64    │
│ 5   │ MARRIAGE                   │ 1.55727   │ 1         │ 2.0      │ 3         │         │          │ Int64    │
│ 6   │ AGE                        │ 35.4855   │ 21      

In [15]:
cc = join(cc, M2, on = :MARRIAGE => :keys)

println(describe(cc))

33×8 DataFrame
│ Row │ variable                   │ mean      │ min       │ median   │ max       │ nunique │ nmissing │ eltype   │
│     │ Symbol                     │ Float64   │ Real      │ Float64  │ Real      │ Nothing │ Nothing  │ DataType │
├─────┼────────────────────────────┼───────────┼───────────┼──────────┼───────────┼─────────┼──────────┼──────────┤
│ 1   │ ID                         │ 15000.5   │ 1         │ 15000.5  │ 30000     │         │          │ Int64    │
│ 2   │ LIMIT_BAL                  │ 1.67484e5 │ 10000.0   │ 140000.0 │ 1.0e6     │         │          │ Float64  │
│ 3   │ SEX                        │ 0.603733  │ 0         │ 1.0      │ 1         │         │          │ Int64    │
│ 4   │ EDUCATION                  │ 1.85377   │ 1         │ 2.0      │ 5         │         │          │ Int64    │
│ 5   │ MARRIAGE                   │ 1.55727   │ 1         │ 2.0      │ 3         │         │          │ Int64    │
│ 6   │ AGE                        │ 35.4855   │ 21      

#### Dropping & Re-ordering Columns

By dropping the columns `EDUCATION1`, `MARRIAGE1` the reference group becomes "Male, Grad School, Married".

In [16]:
keepcols = [:ID, 
            :LIMIT_BAL, 
            :SEX,
            :AGE,
            :EDUCATION2,
            :EDUCATION3,
            :EDUCATION4,
            :EDUCATION5,
            :MARRIAGE2,
            :MARRIAGE3,
            :PAY_0,
            :PAY_2,
            :PAY_3,
            :PAY_4,
            :PAY_5,
            :PAY_6,
            :BILL_AMT1,
            :BILL_AMT2,
            :BILL_AMT3,
            :BILL_AMT4,
            :BILL_AMT5,
            :BILL_AMT6,
            :PAY_AMT1,
            :PAY_AMT2,
            :PAY_AMT3,
            :PAY_AMT4,
            :PAY_AMT5,
            :PAY_AMT6,
            Symbol("default.payment.next.month")]

cc = cc[keepcols]

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[16]:30
└ @ Core In[16]:30


,ID,LIMIT_BAL,SEX,AGE,EDUCATION2,EDUCATION3,EDUCATION4,EDUCATION5
,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,20000.0,1,24,1,0,0,0
2,2,120000.0,1,26,1,0,0,0
3,3,90000.0,1,34,1,0,0,0
4,4,50000.0,1,37,1,0,0,0
5,5,50000.0,0,57,1,0,0,0
6,6,50000.0,0,37,0,0,0,0
7,7,500000.0,0,29,0,0,0,0
8,8,100000.0,1,23,1,0,0,0
9,9,140000.0,1,28,0,1,0,0


In [17]:
cc[:AGE] = Float64.(cc[:AGE])

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[17]:1
└ @ Core In[17]:1
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[17]:1
└ @ Core In[17]:1


30000-element Array{Float64,1}:
 24.0
 26.0
 34.0
 37.0
 57.0
 37.0
 29.0
 23.0
 28.0
 35.0
 34.0
 51.0
 41.0
  ⋮  
 34.0
 35.0
 41.0
 34.0
 43.0
 38.0
 34.0
 39.0
 43.0
 37.0
 41.0
 46.0

## Train & Test Splitting

To make stuff easier later, I split the dataframe into training and test sets, and write them to two separate CSV's

In [18]:
N = size(cc)[1]
inds  = 1:N
test  = sample(inds, Int(0.30*N), replace=false)
train = setdiff(inds, test)
shuffle!(train)

21000-element Array{Int64,1}:
  1955
 17385
  2966
 14246
 28096
 19513
 20251
 17299
 26655
 19718
 25816
 11153
 16288
     ⋮
 28824
 18331
 22009
  3855
 26345
  6860
  9088
 10690
 17626
 20542
 19357
 26915

In [19]:
Set(∪(test, train)) == Set(inds)

true

In [20]:
Xytrain = cc[train, :]

,ID,LIMIT_BAL,SEX,AGE,EDUCATION2,EDUCATION3,EDUCATION4,EDUCATION5
,Int64,Float64,Int64,Float64,Int64,Int64,Int64,Int64
1,1955,240000.0,1,41.0,1,0,0,0
2,17385,180000.0,1,33.0,0,0,0,0
3,2966,50000.0,1,33.0,1,0,0,0
4,14246,210000.0,1,36.0,1,0,0,0
5,28096,150000.0,1,41.0,1,0,0,0
6,19513,260000.0,1,31.0,0,0,0,0
7,20251,100000.0,1,52.0,1,0,0,0
8,17299,150000.0,1,34.0,1,0,0,0
9,26655,50000.0,0,24.0,0,0,0,0


In [21]:
Xytest = cc[test, :]

,ID,LIMIT_BAL,SEX,AGE,EDUCATION2,EDUCATION3,EDUCATION4,EDUCATION5
,Int64,Float64,Int64,Float64,Int64,Int64,Int64,Int64
1,11338,120000.0,0,38.0,0,0,0,0
2,5156,80000.0,0,27.0,1,0,0,0
3,4811,300000.0,1,31.0,0,0,0,0
4,19155,230000.0,1,37.0,1,0,0,0
5,2535,50000.0,0,26.0,1,0,0,0
6,13735,60000.0,1,46.0,1,0,0,0
7,24672,390000.0,0,34.0,1,0,0,0
8,6365,30000.0,1,59.0,0,1,0,0
9,6288,200000.0,1,33.0,0,0,0,0


In [22]:
println(describe(Xytrain))

29×8 DataFrame
│ Row │ variable                   │ mean      │ min       │ median   │ max       │ nunique │ nmissing │ eltype   │
│     │ Symbol                     │ Float64   │ Real      │ Float64  │ Real      │ Nothing │ Nothing  │ DataType │
├─────┼────────────────────────────┼───────────┼───────────┼──────────┼───────────┼─────────┼──────────┼──────────┤
│ 1   │ ID                         │ 15031.6   │ 1         │ 15118.5  │ 30000     │         │          │ Int64    │
│ 2   │ LIMIT_BAL                  │ 1.67741e5 │ 10000.0   │ 140000.0 │ 800000.0  │         │          │ Float64  │
│ 3   │ SEX                        │ 0.604476  │ 0         │ 1.0      │ 1         │         │          │ Int64    │
│ 4   │ AGE                        │ 35.383    │ 21.0      │ 34.0     │ 75.0      │         │          │ Float64  │
│ 5   │ EDUCATION2                 │ 0.467524  │ 0         │ 0.0      │ 1         │         │          │ Int64    │
│ 6   │ EDUCATION3                 │ 0.163857  │ 0       

## Centering and Scaling

Because some of the variables take only positive values and/or have very large values, I perform centering and scaling on some of the variables. 

This allows for easier interpretation of the coefficients and the odds-ration later.

In [23]:
centerscalecols = [:LIMIT_BAL,
                   :AGE,
                   :BILL_AMT1,
                   :BILL_AMT2,
                   :BILL_AMT3,
                   :BILL_AMT4,
                   :BILL_AMT5,
                   :BILL_AMT6,
                   :PAY_AMT1,
                   :PAY_AMT2,
                   :PAY_AMT3,
                   :PAY_AMT4,
                   :PAY_AMT5,
                   :PAY_AMT6]

14-element Array{Symbol,1}:
 :LIMIT_BAL
 :AGE      
 :BILL_AMT1
 :BILL_AMT2
 :BILL_AMT3
 :BILL_AMT4
 :BILL_AMT5
 :BILL_AMT6
 :PAY_AMT1 
 :PAY_AMT2 
 :PAY_AMT3 
 :PAY_AMT4 
 :PAY_AMT5 
 :PAY_AMT6 

The mean and standard deviation is estimated from the training set, and these values are used to standardize both the training and the test sets. The estimates of the mean and stddev are stored in a separate dataframe. 

In [24]:
mu_sig = DataFrame(var=String[], mu=Float64[], std=Float64[])

for i in centerscalecols
    mu  = mean(Xytrain[i])
    sig = std(Xytrain[i])
    push!(mu_sig, [String(i), mu, sig])
    Xytrain[i] = (Xytrain[i] .- mu)./sig
    Xytest[i] = (Xytest[i] .- mu)./sig
end

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[24]:4
└ @ Core ./In[24]:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[24]:5
└ @ Core ./In[24]:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[24]:7
└ @ Core ./In[24]:7
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[24]:7
└ @ Core ./In[24]:7
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[24]:8
└ @ Core ./In[24]:8
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind

In [25]:
mu_sig

,var,mu,std
,String,Float64,Float64
1,LIMIT_BAL,1.67741e5,1.30153e5
2,AGE,35.383,9.19344
3,BILL_AMT1,50979.6,72864.5
4,BILL_AMT2,48912.4,70396.1
5,BILL_AMT3,46942.4,69285.7
6,BILL_AMT4,43314.5,63918.1
7,BILL_AMT5,40308.6,59989.0
8,BILL_AMT6,38981.7,59045.3
9,PAY_AMT1,5637.46,16814.1


In [26]:
println(describe(Xytrain))

29×8 DataFrame
│ Row │ variable                   │ mean         │ min       │ median    │ max     │ nunique │ nmissing │ eltype   │
│     │ Symbol                     │ Float64      │ Real      │ Float64   │ Real    │ Nothing │ Nothing  │ DataType │
├─────┼────────────────────────────┼──────────────┼───────────┼───────────┼─────────┼─────────┼──────────┼──────────┤
│ 1   │ ID                         │ 15031.6      │ 1         │ 15118.5   │ 30000   │         │          │ Int64    │
│ 2   │ LIMIT_BAL                  │ 2.36848e-18  │ -1.21196  │ -0.213141 │ 4.85781 │         │          │ Float64  │
│ 3   │ SEX                        │ 0.604476     │ 0         │ 1.0       │ 1       │         │          │ Int64    │
│ 4   │ AGE                        │ -2.90984e-16 │ -1.56448  │ -0.150433 │ 4.30927 │         │          │ Float64  │
│ 5   │ EDUCATION2                 │ 0.467524     │ 0         │ 0.0       │ 1       │         │          │ Int64    │
│ 6   │ EDUCATION3                 │ 0.16

In [27]:
println(describe(Xytest))

29×8 DataFrame
│ Row │ variable                   │ mean        │ min       │ median    │ max     │ nunique │ nmissing │ eltype   │
│     │ Symbol                     │ Float64     │ Real      │ Float64   │ Real    │ Nothing │ Nothing  │ DataType │
├─────┼────────────────────────────┼─────────────┼───────────┼───────────┼─────────┼─────────┼──────────┼──────────┤
│ 1   │ ID                         │ 14927.9     │ 10        │ 14759.5   │ 29999   │         │          │ Int64    │
│ 2   │ LIMIT_BAL                  │ -0.00657212 │ -1.21196  │ -0.213141 │ 6.39446 │         │          │ Float64  │
│ 3   │ SEX                        │ 0.602       │ 0         │ 1.0       │ 1       │         │          │ Int64    │
│ 4   │ AGE                        │ 0.0371642   │ -1.56448  │ -0.150433 │ 4.74436 │         │          │ Float64  │
│ 5   │ EDUCATION2                 │ 0.468       │ 0         │ 0.0       │ 1       │         │          │ Int64    │
│ 6   │ EDUCATION3                 │ 0.164       

# Saving the Processed Data

The parameter estimates, train set and test set are saved to CSV's.

In [28]:
CSV.write("Data/CCDataCleanTrain.csv", Xytrain)

"Data/CCDataCleanTrain.csv"

In [29]:
CSV.write("Data/CCDataCleanTest.csv", Xytest)

"Data/CCDataCleanTest.csv"

In [30]:
CSV.write("Data/MuSigmas.csv", mu_sig)

"Data/MuSigmas.csv"